In [108]:
from time import time
import pandas as pd
import numpy as np
from gensim.models import KeyedVectors
import re
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns

import itertools
import datetime

from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from keras.layers import Input, Embedding, LSTM, Lambda
import keras.backend as K
from keras.optimizers import Adadelta
from keras.callbacks import ModelCheckpoint

/Users/gagicha/anaconda/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [89]:
train_df=pd.read_csv('train.csv')
test_df=pd.read_csv('test.csv')

In [90]:
train_df.head(10)

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0
5,5,11,12,Astrology: I am a Capricorn Sun Cap moon and c...,"I'm a triple Capricorn (Sun, Moon and ascendan...",1
6,6,13,14,Should I buy tiago?,What keeps childern active and far from phone ...,0
7,7,15,16,How can I be a good geologist?,What should I do to be a great geologist?,1
8,8,17,18,When do you use シ instead of し?,"When do you use ""&"" instead of ""and""?",0
9,9,19,20,Motorola (company): Can I hack my Charter Moto...,How do I hack Motorola DCX3400 for free internet?,0


# preprocessing steps, convert text into list of words

In [28]:
def text_to_word_list(text):
    ''' Pre process and convert texts to a list of words '''
    text = str(text)
    text = text.lower()

    # Clean the text
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "cannot ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r":", " : ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r" u s ", " american ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(r"\s{2,}", " ", text)

    text = text.split()

    return text

# creating vocabulary and inverse vocabulary

vocab is a dict, with keys as the words and the values being the index of those words in the emedding matrix.


In [37]:
vocab={}
inverse_vocab=['<UNK>']

In [38]:
sen='my name is apple'
for word in text_to_word_list(sen):
    if word not in vocab:
        vocab[word]=len(inverse_vocab)
        inverse_vocab.append(word)

In [39]:
vocab

{'apple': 4, 'is': 3, 'my': 1, 'name': 2}

In [41]:
inverse_vocab

['<UNK>', 'my', 'name', 'is', 'apple']

In [ ]:
EMBEDDING_FILE = 'GoogleNews-vectors-negative300.bin.gz'
# MODEL_SAVING_DIR = '/home/ecohen/HDD/HDD4/Models/Kaggle/Quora/'

In [16]:
word2vec= KeyedVectors.load_word2vec_format(EMBEDDING_FILE, binary=True)

# google word2vec model has 3M words in vocab with word vect size 300.

In [20]:
len(word2vec.vocab)

3000000

In [22]:
this=word2vec.word_vec('this')

In [24]:
len(this)

300

# creating vocab and inverse_vocab and sustituting thhe questions to vectors

In [91]:
stops = stopwords.words('english')
# vocab={}
# inverse_vocab=['<UNK>']

# questions=['question1', 'question2']
# for data in [train_df, test_df]:
#     for index, row in data.iterrows():
#         for ques in questions:
#             q2n=[] #question to number representation
#             for word in text_to_word_list(row[ques]):
#                 if word in stops and word not in word2vec.vocab:
#                     continue # continue gives the control to the top of the loop and ignores the remaining statements.
#                 if word not in vocab :
#                     vocab[word]= len(inverse_vocab)
#                     q2n.append(len(inverse_vocab))
#                     inverse_vocab.append(word)
#                 else:
#                     q2n.append(vocab[word])
                    
#             data.set_value(index, ques, q2n)

vocabulary = dict()
inverse_vocabulary = ['<unk>']  # '<unk>' will never be used, it is only a placeholder for the [0, 0, ....0] embedding
word2vec = KeyedVectors.load_word2vec_format(EMBEDDING_FILE, binary=True)

questions_cols = ['question1', 'question2']

# Iterate over the questions only of both training and test datasets
for dataset in [train_df, test_df]:
    for index, row in dataset.iterrows():

        # Iterate through the text of both questions of the row
        for question in questions_cols:

            q2n = []  #question to number representation
            for word in text_to_word_list(row[question]):

                # Check for unwanted words
                if word in stops or word not in word2vec.vocab:
                    continue # continue gives the control to the top of the loop and ignores the remaining statements.

                if word not in vocabulary:
                    vocabulary[word] = len(inverse_vocabulary)
                    q2n.append(len(inverse_vocabulary))
                    inverse_vocabulary.append(word)
                else:
                    q2n.append(vocabulary[word])

            # Replace questions with number representation
            dataset.set_value(index, question, q2n)

In [93]:
train_df.head()

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,"[1, 1, 2, 3, 4, 5, 6]","[1, 1, 2, 3, 4, 5]",0
1,1,3,4,"[7, 8, 9, 10]","[11, 12, 13, 14, 15, 8, 9, 10, 16]",0
2,2,5,6,"[17, 18, 19, 20, 21, 22]","[19, 18, 23, 24, 25]",0
3,3,7,8,"[26, 27, 28]","[29, 30, 31, 32, 31, 33]",0
4,4,9,10,"[34, 35, 36, 37, 38, 39, 40, 41, 42]","[43, 11, 44, 38, 36]",0


In [94]:
train_df.to_csv('train_vectors.csv', index= None, header=True)
test_df.to_csv('test_vectors.csv', index= None, header=True)

In [95]:
test_df.head()

,test_id,question1,question2
0,0,"[1440, 1047, 609, 102, 4347, 1047]","[3418, 252, 1362, 10888, 1362, 5463, 1191, 144..."
1,1,"[784, 9406, 687, 155, 11, 431]","[155, 431, 784, 9406, 1626]"
2,2,"[139, 186, 917, 187, 671, 101]","[917, 187, 671]"
3,3,"[672, 23753]",[300]
4,4,"[1032, 1017]","[1032, 1017]"


In [96]:
len(vocabulary)

58563

In [97]:
vocabulary

{'step': 1,
 'guide': 2,
 'invest': 3,
 'share': 4,
 'market': 5,
 'india': 6,
 'story': 7,
 'koh': 8,
 'noor': 9,
 'diamond': 10,
 'would': 11,
 'happen': 12,
 'indian': 13,
 'government': 14,
 'stole': 15,
 'back': 16,
 'increase': 17,
 'speed': 18,
 'internet': 19,
 'connection': 20,
 'using': 21,
 'vpn': 22,
 'increased': 23,
 'hacking': 24,
 'dns': 25,
 'mentally': 26,
 'lonely': 27,
 'solve': 28,
 'find': 29,
 'remainder': 30,
 'math': 31,
 '^': 32,
 'divided': 33,
 'one': 34,
 'dissolve': 35,
 'water': 36,
 'sugar': 37,
 'salt': 38,
 'methane': 39,
 'carbon': 40,
 'di': 41,
 'oxide': 42,
 'fish': 43,
 'survive': 44,
 'astrology': 45,
 'sun': 46,
 'cap': 47,
 'moon': 48,
 'rising': 49,
 'say': 50,
 'triple': 51,
 'ascendant': 52,
 'buy': 53,
 'keeps': 54,
 'childern': 55,
 'active': 56,
 'far': 57,
 'phone': 58,
 'video': 59,
 'games': 60,
 'good': 61,
 'geologist': 62,
 'great': 63,
 'use': 64,
 'instead': 65,
 'motorola': 66,
 'company': 67,
 'hack': 68,
 'charter': 69,
 'free'

# create embedding matrix

In [98]:
embedding_dim=300
embeddings=np.random.randn(len(vocabulary)+1, embedding_dim)
embeddings[0]=0

# build the embedding matrix
for word, index in vocabulary.items():
    embeddings[index]=word2vec.word_vec(word)
    
del word2vec

In [99]:
train_df.shape

(404290, 6)

# prepare training and validation data

In [109]:
max_seq_length= max(train_df.question1.map(lambda x: len(x)).max(),
                    train_df.question2.map(lambda x: len(x)).max(),
                    test_df.question1.map(lambda x: len(x)).max(),
                    test_df.question2.map(lambda x: len(x)).max(),)

x=train_df[questions_cols]
y=train_df['is_duplicate']
X_train, X_validation, Y_train, Y_validation= train_test_split(x, y, test_size=0.2) 

# Split to dicts
X_train = {'left': X_train.question1, 'right': X_train.question2}
X_validation = {'left': X_validation.question1, 'right': X_validation.question2}
X_test = {'left': test_df.question1, 'right': test_df.question2}

# Convert labels to their numpy representations
Y_train = Y_train.values
Y_validation = Y_validation.values

# Zero padding
for dataset, side in itertools.product([X_train, X_validation], ['left', 'right']):
    dataset[side] = pad_sequences(dataset[side], maxlen=max_seq_length)
    
# Make sure everything is ok
assert X_train['left'].shape == X_train['right'].shape
assert len(X_train['left']) == len(Y_train)

# build the model

In [ ]:
# Model variables
n_hidden = 50
gradient_clipping_norm = 1.25
batch_size = 64
n_epoch = 25

def exponent_neg_manhattan_distance(left, right):
    ''' Helper function for the similarity estimate of the LSTMs outputs'''
    return K.exp(-K.sum(K.abs(left-right), axis=1, keepdims=True))

# The visible layer
left_input = Input(shape=(max_seq_length,), dtype='int32')
right_input = Input(shape=(max_seq_length,), dtype='int32')

embedding_layer = Embedding(len(embeddings), embedding_dim, weights=[embeddings], input_length=max_seq_length, trainable=False)

# Embedded version of the inputs
encoded_left = embedding_layer(left_input)
encoded_right = embedding_layer(right_input)

# Since this is a siamese network, both sides share the same LSTM
shared_lstm = LSTM(n_hidden)

left_output = shared_lstm(encoded_left)
right_output = shared_lstm(encoded_right)

# Calculates the distance as defined by the MaLSTM model
malstm_distance = Lambda(function=lambda x: exponent_neg_manhattan_distance(x[0], x[1]),output_shape=lambda x: (x[0][0], 1))([left_output, right_output])

# Pack it all up into a model
malstm = Model([left_input, right_input], [malstm_distance])

# Adadelta optimizer, with gradient clipping by norm
optimizer = Adadelta(clipnorm=gradient_clipping_norm)

malstm.compile(loss='mean_squared_error', optimizer=optimizer, metrics=['accuracy'])

# Start training
training_start_time = time()

malstm_trained = malstm.fit([X_train['left'], X_train['right']], Y_train, batch_size=batch_size, nb_epoch=n_epoch,
                            validation_data=([X_validation['left'], X_validation['right']], Y_validation))

print("Training time finished.\n{} epochs in {}".format(n_epoch, datetime.timedelta(seconds=time()-training_start_time)))

# plotting results

In [ ]:
# Plot accuracy
plt.plot(malstm_trained.history['acc'])
plt.plot(malstm_trained.history['val_acc'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

# Plot loss
plt.plot(malstm_trained.history['loss'])
plt.plot(malstm_trained.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper right')
plt.show()